# Progress report

### Project introduction: 
Our data science term project aims to model the impact of COVID-19 in Chicago areas, using sociodemographic data (age, income, education etc.) and COVID-19 statistics (hospitalizations, deaths etc.). The sociodemographic and COVID-19 data used here is for each zipcode within Chicago. An existing COVID-19 model, the CCVI score, ranks neighborhoods in Chicago by vulnerability based on several sociodemographic factors, and develops an overall vulnerability score for each neighborhood based on these rankings. An issue with this ranking system is that the difference in factors is not accurately reflected by a difference in ranks; for example, two neighborhoods could have a relatively big difference in income but only be a rank apart, and vice versa (relatively small difference in income and a rank apart). The ranking-based score would normalize all differences in sociodemographic factors when calculating the vulnerability score, which could be a potential issue when deciding on how to accurately and precisely distribute resources to areas based on their COVID-19 vulnerability. Our model seeks to improve upon the CCVI score by assessing areas in a more absolute way, such that it can predict COVID-19 risk of an area in terms of cases, hospitalizations or deaths, given the sociodemographic factors of that area. 
We chose to develop the model for Chicago both because of our proximity to the city, and because Chicago has been a big COVID-19 hotspot in America during the last two years. Being a big, densely populated city, Chicago has had 567,470 COVID-19 cases, 41,694 hospitalizations and 7,635 deaths as of today. Despite these high numbers, the impact of COVID-19 has been unevenly distributed over the city of Chicago, and is primarily dictated by the sociodemographic factors that we are inspecting in our project. Since our model aims to predict COVID-19 impact from sociodemographic data, it can potentially help organizations distribute resources, healthcare and vaccination efforts in the city more accurately, as well as help policy-makers understand how the pandemic impact is affected by disparities in the city, and plan accordingly in the future to remedy this.


## 2. Any changes
 a discussion whether your scope has changed since the check-in proposal slides. What did you aim to do that you will not do and what have you added to the project?

## 3. Data cleaning: 
show clearly how you cleaned your data. 

## 4. Exploratory data analysis: 
explain what your data looks like (words are fine, but visualizations are often better). Include any interesting issues or preliminary conclusions you have about your data. 

## 5. At least one visualization 
that tests an interesting hypothesis, along with an explanation about why you thought this was an interesting hypothesis to investigate.  

## 6. At least one ML analysis
on your dataset, along with a baseline comparison and an interpretation of the result that you obtain. 

## 7. Reflection: 
a discussion of the following: 
    o What is hardest part of the project that you’ve encountered so far? 
    o What are your initial insights? 
    o Are there any concrete results you can show at this point? If not, why not? 
    o Going forward, what are the current biggest problems you’re facing? 
    o Do you think you are on track with your project? If not, what parts do you need to dedicate more time to? 
    o Given your initial exploration of the data, is it worth proceeding with your project, why? If not, how are you going to change your project and why do you think it’s better than your current results? 

## 8. Next steps: 
What you plan to accomplish in the next month and how you plan to evaluate whether your project achieved the goals you set for it. 